In [1]:
# Adapted from https://www.kaggle.com/code/ruchitass/predicting-stress-a-machine-learning-approach

In [2]:
!python --version

Python 3.9.19


In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re

# Download the stopwords resource
nltk.download('stopwords')

stemmer = nltk.SnowballStemmer("english")

from wordcloud import STOPWORDS
from PIL import Image
from wordcloud import WordCloud
import pickle

# from textblob import TextBlob
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# import nltk
# import re
# stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
# import string

import matplotlib.pyplot as plt
import kaggle


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# import warnings
# warnings.filterwarnings("ignore")

import string
#Get a list of punctuations
punct = []
for char in string.punctuation:
    punct.append(char)

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [5]:
# import xgboost

In [6]:
import mlflow
import os

#os.environ["AWS_PROFILE"] = "dara" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-13-50-246-102.eu-north-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
mlflow.__version__

'2.12.2'

In [8]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-13-50-246-102.eu-north-1.compute.amazonaws.com:5000'


In [9]:
train = pd.read_csv("../../data/dreaddit-train.csv")
test = pd.read_csv("../../data/dreaddit-test.csv")

In [10]:
numerical_columns = ["lex_liwc_Tone", "lex_liwc_i", "lex_liwc_negemo", "lex_liwc_Clout", "sentiment"]

In [11]:
numerical_columns 

['lex_liwc_Tone',
 'lex_liwc_i',
 'lex_liwc_negemo',
 'lex_liwc_Clout',
 'sentiment']

In [12]:
# Standardization
scaler = StandardScaler()
# X_numerical = scaler.fit_transform(train_numerical)

In [13]:
def process_numerical_features(df):
    X_numerical = df[numerical_columns]
    X_numerical = scaler.fit_transform(train_numerical)
    return X_numerical
    

In [14]:
def removal(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [w for w in text.split(' ') if w not in stopwords]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [15]:

vect=CountVectorizer(stop_words="english")

In [16]:
def process_categorical_features(df, vect = None):
    posts = train[["text"]]
    posts["text"] = posts["text"].apply(removal)
    # posts["sentiment"] = posts["text"].apply(mood)
    X = posts["text"]
    if vect:
        print("here")
        X = vect.transform(X)
    else:
        vect=CountVectorizer(stop_words="english")
        X=vect.fit_transform(X)
    return X, vect
    

In [17]:
def prepare_features(df, vect=None):
    X_categorical, vect = process_categorical_features(train, vect)
    X_numerical = process_numerical_features(train)
    X_combined = np.hstack((X_categorical.toarray(), X_numerical))

    return X_combined, vect
    
    

In [18]:
train_numerical = train[numerical_columns] 

In [19]:
X_combined, vect = prepare_features(train)

In [39]:
directory = 'models'
os.makedirs(directory, exist_ok=True)

with open("models/vectorizer.b", "wb") as f_out:
    pickle.dump(vect, f_out)


In [20]:
X_combined.shape;

In [21]:
X_text_features, _ = process_categorical_features(train)
X_text_features.toarray().shape

(2838, 9448)

In [22]:
y = train["label"]

In [23]:
x_train_combined,x_test_combined,y_train,y_test=train_test_split(X_combined,y,random_state=43)
# x_train,x_test,y_train,y_test=train_test_split(train[numerical_columns + categorical_columns],y,random_state=43)

In [24]:
X_combined.shape

(2838, 9453)

In [25]:
# X_text_features

In [26]:
x_train_text,x_test_text,y_train,y_test=train_test_split(X_text_features.toarray(),y,random_state=43)

In [27]:
x_train_combined;

In [28]:
# experiment_name = "project-experiment-tracking"
# mlflow.set_experiment(experiment_name)

In [29]:
# experiment = mlflow.get_experiment_by_name(experiment_name)

In [30]:
# experiment_id = experiment.experiment_id

In [31]:
# experiment_id

    
    

In [32]:
# mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")

    

<Experiment: artifact_location='s3://mlflows-artifacts-remote/2', creation_time=1720812846024, experiment_id='2', last_update_time=1720812846024, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>

### Hyperparameter tuning

In [33]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope


def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("developer", "Dara")

        # Log hyperparameters
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)

        rf = RandomForestClassifier(**params)
        rf.fit(x_train_combined, y_train)
        y_pred = rf.predict(x_test_combined)
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

    return {'loss': -accuracy, 'status': STATUS_OK}  # Negative because fmin minimizes

def run_optimization(num_trials=15):
    search_space = {
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 200, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 5, 1)),
        'random_state': 42
    }


    rstate = np.random.default_rng(42)  # for reproducible results
    trials = Trials()
    best = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=trials,
        rstate=rstate
    )

    print("Best hyperparameters: ", best)

# Run optimization
run_optimization(num_trials=15)



100%|██████████████████████████| 15/15 [01:15<00:00,  5.05s/trial, best loss: -0.747887323943662]
Best hyperparameters:  {'min_samples_split': 2.0, 'n_estimators': 106.0}


### Model Registry

In [51]:
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient


HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"
RF_PARAMS_INT = ['n_estimators', 'min_samples_split', 'random_state']
RF_PARAMS_BOOL = ['bootstrap','oob_score','warm_start']

mlflow.set_experiment(EXPERIMENT_NAME)
#To log other interesting metrics
mlflow.sklearn.autolog()

def train_and_log_model(params):
    # print(params)
    new_params = {}
    with mlflow.start_run():
        for param in RF_PARAMS_INT:
            new_params[param] = int(params[param])
        
        rf = RandomForestClassifier(**new_params)
        rf.fit(x_train_combined, y_train)
        mlflow.sklearn.log_model(rf, f"model")
        
        #Log vecorizer for the text dat
        with open("models/vectorizer.b", "wb") as f_out:
            pickle.dump(vect, f_out)
        
        mlflow.log_artifact("models/vectorizer.b", artifact_path="vectorizer")
            
        # Evaluate model on the and test sets
        accuracy = round(accuracy_score(y_test, rf.predict(x_test_combined))*100,2)
        mlflow.log_metric("accuracy", accuracy)


def run_register_model(top_n: int):

    client = MlflowClient()

    # Retrieve the top_n model runs and log the models
    experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=top_n,
        order_by=["metrics.accuracy DESC"]
    )

    for run in runs:
        params=run.data.params
        train_and_log_model(params=run.data.params)

    # Select the model with the lowest best accuracy
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    # best_run = client.search_runs(experiment_ids="1")[0]
    best_run = client.search_runs(experiment_ids=[experiment.experiment_id], max_results=top_n, order_by=["metrics.accuracy DESC"])
    
    # Register the best model
    run_id = best_run[0].info.run_id
    model_uri = f'runs:/{run_id}/model'
    print(run_id)
    mlflow.register_model(model_uri=model_uri, name="stress_predictor")
    print("registered successfully")

    return client, run_id



In [52]:
client, run_id = run_register_model(5)

2024/07/13 10:28:47 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2024/07/13 10:29:00 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2024/07/13 10:29:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2024/07/13 10:29:29 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2024/07/13 10:29:40 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
Registered model 'stress_predictor' already exists. Creating a new version of this model...
2024/07/13 10:29:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to fi

57342ae687254eeeac28602bb8d42aca
registered successfully


Created version '4' of model 'stress_predictor'.


In [53]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
experiment_id = experiment.experiment_id

In [55]:
model_name = "stress_predictor"
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
)

<ModelVersion: aliases=[], creation_timestamp=1720865831915, current_stage='Staging', description='The model version 3 was transitioned to Production on 2024-07-13', last_updated_timestamp=1720866644723, name='stress_predictor', run_id='49b5483feda54e1482a9cb3ed924d823', run_link='', source='s3://mlflows-artifacts-remote/3/49b5483feda54e1482a9cb3ed924d823/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [56]:
from datetime import datetime

model_version= 3
new_stage = "Production"
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
   )

<ModelVersion: aliases=[], creation_timestamp=1720865831915, current_stage='Staging', description='The model version 3 was transitioned to Production on 2024-07-13', last_updated_timestamp=1720866655333, name='stress_predictor', run_id='49b5483feda54e1482a9cb3ed924d823', run_link='', source='s3://mlflows-artifacts-remote/3/49b5483feda54e1482a9cb3ed924d823/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [61]:
#A way to load data
# logged_model = f'https://mlflows-artifacts-remote.s3.eu-north-1.amazonaws.com/{experiment_id}/{run_id}/'
logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 57342ae687254eeeac28602bb8d42aca

In [64]:
predictions = loaded_model.predict(x_test_combined)

In [65]:
predictions

array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,

In [67]:
#Load vectorizer
client.download_artifacts(run_id=run_id, path='vectorizer', dst_path='.')

'/home/ubuntu/mental_health_mlops_project/model/experiment_tracking/vectorizer'

In [68]:
with open("vectorizer/vectorizer.b", "rb") as f_in:
    vect = pickle.load(f_in)

In [69]:
vect

CountVectorizer(stop_words='english')